In [1]:
from __future__ import absolute_import, division, print_function

# Import TensorFlow >= 1.10 and enable eager execution
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences
from pprint import pprint

tf.enable_eager_execution()
tf.set_random_seed(777)

/Users/sinseongjin/tf110/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


# Preparing Dataset

In [2]:
words = ['good', 'bad', 'worse', 'so good']
y_data = [1,0,0,1]
char_set = ['<pad>'] + sorted(list(set(''.join(words))))
idx2char = {idx : char for idx, char in enumerate(char_set)}
char2idx = {char : idx for idx, char in enumerate(char_set)}

print(idx2char)
print(char2idx)

{0: '<pad>', 1: ' ', 2: 'a', 3: 'b', 4: 'd', 5: 'e', 6: 'g', 7: 'o', 8: 'r', 9: 's', 10: 'w'}
{'<pad>': 0, ' ': 1, 'a': 2, 'b': 3, 'd': 4, 'e': 5, 'g': 6, 'o': 7, 'r': 8, 's': 9, 'w': 10}


In [3]:
x_data = list(map(lambda word : [char2idx.get(char) for char in word], words))
x_data_len = list(map(lambda word : len(word), x_data))

print(x_data)
print(x_data_len)

[[6, 7, 7, 4], [3, 2, 4], [10, 7, 8, 9, 5], [9, 7, 1, 6, 7, 7, 4]]
[4, 3, 5, 7]


In [4]:
# padding
max_sequence = 10
x_data = pad_sequences(sequences = x_data, maxlen = max_sequence,
                       padding = 'post', truncating = 'post')

# 데이터 형태 확인
print(x_data)
print(x_data_len)
print(y_data)

[[ 6  7  7  4  0  0  0  0  0  0]
 [ 3  2  4  0  0  0  0  0  0  0]
 [10  7  8  9  5  0  0  0  0  0]
 [ 9  7  1  6  7  7  4  0  0  0]]
[4, 3, 5, 7]
[1, 0, 0, 1]


In [5]:
# input
data = tf.data.Dataset.from_tensor_slices((x_data_len, x_data, y_data))
data = data.shuffle(buffer_size = 4)
data = data.batch(batch_size = 2)

In [16]:
for x_len, x_data, y_data in data:
    a = x_data

In [7]:
# idx2char
n_of_classes = len(idx2char)
hidden_size = 10

In [13]:
def gru(units):
  # If you have a GPU, we recommend using CuDNNGRU(provides a 3x speedup than GRU)
  # the code automatically does that.
    if tf.test.is_gpu_available():
        return tf.keras.layers.CuDNNGRU(units, 
                                        return_sequences=True, 
                                        return_state=True, 
                                        recurrent_initializer='glorot_uniform')
    else:
        return tf.keras.layers.GRU(units, 
                                   return_sequences=True, 
                                   return_state=True, 
                                   recurrent_activation='sigmoid', 
                                   recurrent_initializer='glorot_uniform')

class Model(tf.keras.Model):
    def __init__(self, n_of_classes, enc_units, batch_size):
        super(Model, self).__init__()
        self.batch_size = batch_size
        self.enc_units = enc_units
        self.embedding = keras.layers.Embedding(n_of_classes, 10)
        self.gru = gru(self.enc_units)
    
    def call(self, x, hidden):
        x = self.embedding(x)
        output, state = self.gru(x, initial_state = hidden)
        score = keras.layers.Dense(units=n_of_classes)(state)
        
        return score
    
    def initialize_hidden_state(self):
        return tf.zeros((self.batch_size, self.enc_units))

In [18]:
rnn_model = Model(n_of_classes, 10, 2)
hidden = rnn_model.initialize_hidden_state()

In [19]:
a=rnn_model(a, hidden)

In [20]:
a

<tf.Tensor: id=4554, shape=(2, 11), dtype=float32, numpy=
array([[ 0.41145578, -0.6191872 , -0.08279973,  0.12691   , -0.4117321 ,
         0.26026696, -0.12846631,  0.3612137 , -0.06026096,  0.00758313,
        -0.09441513],
       [ 0.41145578, -0.6191872 , -0.08279973,  0.12691   , -0.4117321 ,
         0.26026696, -0.12846631,  0.3612137 , -0.06026096,  0.00758313,
        -0.09441513]], dtype=float32)>

In [129]:
score = keras.layers.Dense(units=n_of_classes)(b)
score

<tf.Tensor: id=3717, shape=(2, 11), dtype=float32, numpy=
array([[ 0.15921393, -0.1025836 ,  0.68354   ,  0.21409467,  0.6078851 ,
         0.03582001,  0.07181011,  0.6185158 , -0.5359601 , -0.4140931 ,
        -0.29106885],
       [ 0.17644891, -0.07634956,  0.65046906,  0.19568354,  0.6246023 ,
         0.04107103,  0.08192392,  0.62382454, -0.5419755 , -0.4236086 ,
        -0.2594868 ]], dtype=float32)>

In [9]:
# hyper-parameters for lstm (many to one), one-hot encoding
n_of_classes = len(idx2char)
hidden_size = 10

one_hot_encoding = tf.eye(num_rows = n_of_classes, dtype = tf.float32)
one_hot_encoding = tf.get_variable(name = 'one_hot_encoding',
                                   initializer = one_hot_encoding, trainable = False)

x_batch = tf.nn.embedding_lookup(params = one_hot_encoding, ids = x)

NameError: name 'x' is not defined